<a href="https://colab.research.google.com/github/msrana172/Big-Data-Engineering-Coursera-Yandex/blob/master/mnf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade google-cloud-translate

In [ ]:
import os
from google.cloud import translate_v2 as translate
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/unique-decker-278617-ead19fb5dcc9.json"

translate_client = translate.Client()

#text ="my name is lokesh."
text = "My name is Goat cheese"
target = "fr"

result = translate_client.translate(text, target_language=target)

print(u"Text: {}".format(result["input"]))
print(u"Translation: {}".format(result["translatedText"]))
print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

Text: My name is Goat cheese
Translation: Mon nom est fromage de chèvre
Detected source language: en


In [ ]:
result

{'detectedSourceLanguage': 'en',
 'input': 'my name is lokesh',
 'translatedText': 'मेरा नाम लोकेश है'}

In [ ]:
def create_glossary(languages, project_id, glossary_name, glossary_uri):
    
    timeout=180
    location = "us-central1"

    client = translate.TranslationServiceClient()
    name = client.glossary_path(project_id, location, glossary_name)

  
    language_codes_set = translate.Glossary.LanguageCodesSet(
        language_codes=languages)

    gcs_source = translate.GcsSource(input_uri=glossary_uri)

    input_config = translate.GlossaryInputConfig(gcs_source=gcs_source)

    glossary = translate.Glossary(
        name=name, language_codes_set=language_codes_set, input_config=input_config
    )

    parent = f"projects/{project_id}/locations/{location}"

    #print("hello")
    operation = client.create_glossary(parent=parent, glossary=glossary)
    #print("hello")

    result = operation.result(timeout)
    print("Created: {}".format(result.name))
    print("Input Uri: {}".format(result.input_config.gcs_source.input_uri))

In [ ]:
import os
from google.api_core.exceptions import AlreadyExists
from google.cloud import translate_v3 as translate
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/unique-decker-278617-ead19fb5dcc9.json"
os.environ["GCLOUD_PROJECT"]="unique-decker-278617"


In [ ]:
glossary_langs = ["fr", "en"]
PROJECT_ID = "unique-decker-278617"
glossary_name = "lokeshgupta1"
glossary_uri = "gs://gupta/Book1.csv"
create_glossary(glossary_langs, PROJECT_ID, glossary_name, glossary_uri)

Created: projects/1073430299264/locations/us-central1/glossaries/lokeshgupta1
Input Uri: gs://gupta/Book1.csv


In [ ]:
from google.cloud import translate
def translate_text_with_glossary(
    text = "YOUR_TEXT_TO_TRANSLATE",
    project_id = "YOUR_PROJECT_ID",
    glossary_id = "YOUR_GLOSSARY_ID",
):
    """Translates a given text using a glossary."""

    client = translate.TranslationServiceClient()
    location = "us-central1"
    parent = f"projects/{project_id}/locations/{location}"

    glossary = client.glossary_path(
        project_id, "us-central1", glossary_id  # The location of the glossary
    )

    glossary_config = translate.TranslateTextGlossaryConfig(glossary=glossary)

    # Supported language codes: https://cloud.google.com/translate/docs/languages
    response = client.translate_text(
        request={
            "contents": [text],
            "target_language_code": "fr",
            "source_language_code": "en",
            "parent": parent,
            "glossary_config": glossary_config,
        }
    )

    print("Translated text: \n")
    for translation in response.glossary_translations:
        print("\t {}".format(translation.translated_text))

In [ ]:
translate_text_with_glossary("My name is Goat cheese", "unique-decker-278617", "lokeshgupta" )

Translated text: 

	 Mon nom est Chevre


In [ ]:
Mon nom est lokesh 

In [ ]:
Mon nom est fromage de chèvre

In [ ]:
from google.cloud import translate
def list_glossaries(project_id="YOUR_PROJECT_ID"):
    """List Glossaries."""

    client = translate.TranslationServiceClient()

    location = "us-central1"

    parent = f"projects/{project_id}/locations/{location}"

    # Iterate over all results
    for glossary in client.list_glossaries(parent=parent):
        print("Name: {}".format(glossary.name))
        print("Entry count: {}".format(glossary.entry_count))
        print("Input uri: {}".format(glossary.input_config.gcs_source.input_uri))

        # Note: You can create a glossary using one of two modes:
        # language_code_set or language_pair. When listing the information for
        # a glossary, you can only get information for the mode you used
        # when creating the glossary.
        for language_code in glossary.language_codes_set.language_codes:
            print("Language code: {}".format(language_code))

In [ ]:
list_glossaries("unique-decker-278617")

Name: projects/1073430299264/locations/us-central1/glossaries/gl
Entry count: 11
Input uri: gs://cloud-samples-data/translation/bistro_glossary.csv
Language code: fr
Language code: en
Name: projects/1073430299264/locations/us-central1/glossaries/gloss
Entry count: 11
Input uri: gs://cloud-samples-data/translation/bistro_glossary.csv
Language code: fr
Language code: en
Name: projects/1073430299264/locations/us-central1/glossaries/glossary
Entry count: 11
Input uri: gs://cloud-samples-data/translation/bistro_glossary.csv
Language code: fr
Language code: en
Name: projects/1073430299264/locations/us-central1/glossaries/hmm
Entry count: 8
Input uri: gs://gupta/Book1.csv
Language code: fr
Language code: en
Name: projects/1073430299264/locations/us-central1/glossaries/lok
Entry count: 8
Input uri: gs://gupta/Book1.csv
Language code: fr
Language code: en
Name: projects/1073430299264/locations/us-central1/glossaries/loke
Entry count: 11
Input uri: gs://cloud-samples-data/translation/bistro_glos

In [ ]:
from google.cloud import translate_v3 as translate

def delete_glossary(
    project_id="YOUR_PROJECT_ID", glossary_id="YOUR_GLOSSARY_ID", timeout=180,
):
    """Delete a specific glossary based on the glossary ID."""
    client = translate.TranslationServiceClient()

    name = client.glossary_path(project_id, "us-central1", glossary_id)

    operation = client.delete_glossary(name=name)
    result = operation.result(timeout)
    print("Deleted: {}".format(result.name))


In [ ]:
delete_glossary("unique-decker-278617", "bistro-glossary" )

Deleted: projects/1073430299264/locations/us-central1/glossaries/bistro-glossary


In [ ]:
from google.cloud import translate_v3 as translate
def get_glossary(project_id = "YOUR_PROJECT_ID", glossary_id="YOUR_GLOSSARY_ID"):
    """Get a particular glossary based on the glossary ID."""

    client = translate.TranslationServiceClient()

    name = client.glossary_path(project_id, "us-central1", glossary_id)

    response = client.get_glossary(name=name)
    print(u"Glossary name: {}".format(response.name))
    print(u"Entry count: {}".format(response.entry_count))
    print(u"Input URI: {}".format(response.input_config.gcs_source.input_uri))

In [ ]:
get_glossary("unique-decker-278617", "lokeshgupta")

Glossary name: projects/1073430299264/locations/us-central1/glossaries/lokeshgupta
Entry count: 8
Input URI: gs://gupta/Book1.csv


In [ ]:
glossary_langs = ["fr", "en"]
PROJECT_ID = "unique-decker-278617"
glossary_name = "gl"
glossary_uri = "gs://cloud-samples-data/translation/bistro_glossary.csv"
create_glossary(glossary_langs, PROJECT_ID, glossary_name, glossary_uri)

hello
hello
Created: projects/1073430299264/locations/us-central1/glossaries/gl
Input Uri: gs://cloud-samples-data/translation/bistro_glossary.csv


In [ ]:
glossary_langs = ["fr", "en"]
PROJECT_ID = "unique-decker-278617"
glossary_name = "hmm"
glossary_uri = "gs://gupta/Book1.csv"
create_glossary(glossary_langs, PROJECT_ID, glossary_name, glossary_uri)

hello
hello
Created: projects/1073430299264/locations/us-central1/glossaries/hmm
Input Uri: gs://gupta/Book1.csv
